In [55]:
import requests

def get_nutrients(fdc_ids):
    # Initialize a dictionary to store the results
    results = {}

    # Iterate over the FoodData Central IDs
    for fdc_id in fdc_ids:
        # Define the API endpoint URL
        url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"

        # Define the query parameters
        params = {"api_key": "B2Gehrdg2uN9J0Rk1o8PGrdRIL8ojBo3aaJOjxRb"}

        # Send the GET request
        response = requests.get(url, params=params)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract the JSON data from the response
            data = response.json()

            # Extract the food name
            food_name = data["description"]

            # Initialize the nutrients data dictionary with the food name
            nutrients_data = {"name": food_name}

            # Extract the nutrients data
            for nutrient in data["foodNutrients"]:
                nutrient_name = nutrient["nutrient"]["name"]
                nutrients_data[nutrient_name] = nutrient.get("amount", None)

            # Store the nutrients data in the results dictionary
            results[fdc_id] = nutrients_data
        else:
            print(f"Error: API request failed for FDC ID {fdc_id}")
            print("Status code:", response.status_code)
            print("Response text:", response.text)

    # Return the results
    return results

# Define the FoodData Central IDs
fdc_ids = ["1750340", "2003590"]

# Call the function and store the results
results = get_nutrients(fdc_ids)

Error: API request failed for FDC ID 1750340
Status code: 429
Response text: {
  "error": {
    "code": "OVER_RATE_LIMIT",
    "message": "You have exceeded your rate limit. Try again later or contact us at https://api.nal.usda.gov:443/contact/ for assistance"
  }
}


In [50]:
# Define the FoodData Central IDs
fdc_ids = ["1750340", "2003590"]

# Call the function and store the results
results = get_nutrients(fdc_ids)

# Print the results in a readable format
pprint(results)


Error: API request failed for FDC ID 1750340
Error: API request failed for FDC ID 2003590
{}
